# Usage of the Imagine Model library

In this tutorial, we focus on introducing the model library and the IMAGINE built-in simulators as a building block of the IMAGINE package and how to use it for assembling a Bayesian analysis pipeline.
     

## 0) Preliminaries



### 0.a) First the imports and path settings

In [16]:
import imagine as img
from imagine.simulators.rmlos import RMSimulator
from imagine.fields.library.jf12 import JF12Regular
from imagine.fields.library.ymw16 import YMW16

import numpy as np
from astropy.coordinates import cartesian_to_spherical
import astropy.units as u
import os

# Directory paths
rundir = 'runs/mockdata'
figpath = 'figures/simulator_testing/'
logdir = 'log/'

os.system("rm -r runs/mockdata/*") # important, as multinest loads the previous piepline by default

dunit = u.rad/(u.m**2)


### 0.b) Defining some helper functions for the data generation

The following convenience functions wrap the generation of the mock data set (i.e. RMs, their errors and their position), and the set up of the corresponding IMAGINE observable dicts. 
The reader can skip this cell, for details see [tutorial_datasets](https://github.com/IMAGINE-Consortium/imagine/blob/master/tutorials/tutorial_datasets.ipynb). 

In [17]:
def produce_mock_data(field_list, mea, config, nd, noise=0.01):
    """Runs the simulator once to produce a simulated dataset"""
    test_sim = RMSimulator(measurements=mea, sim_config=config)
    simulation = test_sim(field_list)
    key = ('fd', None, 'tab', None)
    sim_data = simulation[key].data[0] * simulation[key].unit
    sim_data += np.random.normal(loc=0, scale=noise*abs(sim_data), size=nd)*simulation[key].unit
    return sim_data


def build_mock_data_and_config(nth, bfield, cart_grid, n_data):
        # Produce empty data format
    rm = np.zeros(n_data)*dunit  # placeholder
    rm_err = np.zeros(n_data)*dunit  # placeholder

    x = np.random.uniform(-0.9*cart_grid.box[0][0].value, 0.9*cart_grid.box[0][1].value, n_data)*u.kpc
    y = np.random.uniform(-0.9*cart_grid.box[1][0].value, 0.9*cart_grid.box[1][1].value, n_data)*u.kpc
    z = np.random.uniform(-0.1*cart_grid.box[2][0].value, 0.1*cart_grid.box[2][1].value, n_data)*u.kpc
    dist, lat, lon = cartesian_to_spherical(x+8.5*u.kpc, y, z)
    ph_data = {'fd': rm, 'err': rm_err, 'lat': lat, 'lon': lon}
    ph_data = img.observables.TabularDataset(ph_data,
                                               name='fd',
                                               units=dunit,
                                               data_col='fd',
                                               err_col='err',
                                               lat_col='lat',
                                               lon_col='lon')
    data_config =  img.observables.Measurements(ph_data)
    # Setup the Galactic field models


    # Setup observing configuration
    observer = np.array([-8.5, 0, 0])*u.kpc
    dist_err = dist/100000
    config = {'grid': cart_grid,
              'observer': observer,
              'dist': dist,
              'e_dist': dist_err,
              'lat': lat,
              'lon': lon,
              'FB': None}

    # Produce simulated dataset with noise
    noise_level = 0.01
    
    mock_data = produce_mock_data(field_list=[nth, bfield], mea=data_config, config=config, nd=n_data, noise=noise_level)
    
    
    mock_data = {'fd': mock_data, 'err': mock_data*noise_level, 'lat': config['lat'], 'lon': config['lon']}
    mock_data = img.observables.TabularDataset(mock_data,
                                               name='fd',
                                               units=dunit,
                                               data_col='fd',
                                               err_col='err',
                                               lat_col='lat',
                                               lon_col='lon')
    mock_data =  img.observables.Measurements(mock_data)

    
    return data_config, config, mock_data


## 1) Setting up the geometry

Here, we define the grid representing the Galaxy in a cartesian box.  This is done be providing the edges of the box, as well as the resolution.
For details see the [tutorial_fields](https://github.com/IMAGINE-Consortium/imagine/blob/master/tutorials/tutorial_fields.ipynb). 

In [18]:
n_data = 1000 # number of data points

xmax = 15*u.kpc  # size of the box
ymax = 15*u.kpc
zmax = 2*u.kpc

# setting up the 
cartesian_grid = img.fields.UniformGrid(box=[[-xmax, xmax], 
                                            [-ymax, ymax],
                                            [-zmax, zmax]],
                                    resolution=[30, 30, 30])  


## 1) Building the fields, mock data and likelihood

In [19]:

nth = img.fields.ConstantThermalElectrons(grid=cartesian_grid, parameters={'ne': 1*u.cm**(-3)})
# nth = YMW16(grid=cartesian_grid, parameters={'ne': 1*u.cm**(-3)})
bfield = JF12Regular(grid=cartesian_grid) # using default parameters




### 2.b) Building the Likelihood

We use the convenience functions defined above, as well as

In [20]:

data_config, simulator_config, mock_data = build_mock_data_and_config(nth, bfield, cartesian_grid, n_data)    
# Setup simulator
los_simulator = RMSimulator(data_config, simulator_config)
# Initialize likelihood
likelihood = img.likelihoods.SimpleLikelihood(mock_data)



## 3) Building the Field Factories 

Field factories are IMAGINE objects which store the active parameters and the prior information on them. 
For details  

In [21]:

# Setup field factories and their active parameters
nth_factory = img.fields.FieldFactory(field_class=nth, grid=cartesian_grid)

B_factory   = img.fields.FieldFactory(field_class = bfield, grid=cartesian_grid)
B_factory.active_parameters = ('b_arm_1','b_arm_2', 
                               'b_arm_3','b_arm_4', 
                               # 'b_arm_5','b_arm_6', 'b_arm_7'
                               )


# true values are b_arm_1 = 0.1, b_arm_2 = 3.0, b_arm_3 = -0.9, b_arm_4 = -0.8, b_arm_5 = -2.0, b_arm_6 = -4.2, b_arm_7 = 0 

B_factory.priors = {
'b_arm_1':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
'b_arm_2':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
'b_arm_3':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
'b_arm_4':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
#'b_arm_5':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
#'b_arm_6':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
#'b_arm_7':img.priors.GaussianPrior(mu=0*u.microgauss, sigma=6*u.microgauss),
}

factory_list = [B_factory, nth_factory]
# Setup final pipeline 
# los_simulator.fields = {'thermal_electron_density': nth, 'magnetic_field' : bp}


In [22]:

pipeline = img.pipelines.MultinestPipeline(simulator=los_simulator,
                                            run_directory=rundir,
                                            factory_list=factory_list,
                                            likelihood=likelihood)
pipeline.sampling_controllers = {'evidence_tolerance': 0.05, 'n_live_points': 1000}

# Run!
results = pipeline()

summary = pipeline.posterior_summary
samples = pipeline.samples



 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points = 1000
 dimensionality =    4
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.999049
Replacements:                               1050
Total Samples:                              1051
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z):   29224.189609 +/-  0.999524
Acceptance Rate:                        0.997280
Replacements:                               1100
Total Samples:                              1103
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z):   29224.141317 +/-  0.999547
Acceptance Rate:                        0.996534
Replacements:                               1150
